In [1]:
!pip install crewai  crewai_tools langchain-community langchain-openai

In [2]:
import os
import yaml
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI

In [3]:
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

# Loading Agents and Tasks YAML files

In [5]:
# Defining file paths for YAML configurations
files = {
    'lead_agents': './config/lead_qualification_agents.yaml',
    'lead_tasks': './config/lead_qualification_tasks.yaml',
    'email_agents': './config/email_engagement_agents.yaml',
    'email_tasks': './config/email_engagement_tasks.yaml'
}

# Loading configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assigning loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

# Create Pydantic Models for Structured Output

In [6]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")

# Importing Tools

In [7]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

# Lead Qualification Crew, Agents and Tasks

In [8]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
  llm = ChatOpenAI(model_name = "gpt-4o")
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
  llm = ChatOpenAI(model_name = "gpt-4o")
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
  llm = ChatOpenAI(model_name = "gpt-4o")
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)

# E-mail Engagement Crew

In [ ]:
# Creating Agents
email_content_specialist = Agent(
  config=email_agents_config['email_content_specialist'],
  llm = ChatOpenAI(model_name = "gpt-4o")
)

engagement_strategist = Agent(
  config=email_agents_config['engagement_strategist'],
  llm = ChatOpenAI(model_name = "gpt-4o")
)

# Creating Tasks
email_drafting = Task(
  config=email_tasks_config['email_drafting'],
  agent=email_content_specialist
)

engagement_optimization = Task(
  config=email_tasks_config['engagement_optimization'],
  agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
  agents=[
    email_content_specialist,
    engagement_strategist
  ],
  tasks=[
    email_drafting,
    engagement_optimization
  ],
  verbose=True
)

# Creating Complete Sales flow

In [10]:
from crewai import Flow
from crewai.flow.flow import listen, start

class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "Olivia Robinson",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "oliviarb@clearbit.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score['lead_score'].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()

# Plotting The flow

In [11]:
flow.plot()

Plot saved as crewai_flow.html


In [12]:
# from IPython.display import IFrame

# IFrame(src='/content/crewai_flow.html', width='150%', height=600)

from IPython.display import HTML, display

with open('/content/crewai_flow.html', 'r') as f:
    html_content = f.read()

display(HTML(html_content))

# Flow Kickoff

In [17]:
# import nest_asyncio
# nest_asyncio.apply()

emails = flow.kickoff()

 Flow started with ID: 5dcec011-322f-4178-8ed5-858a08055077
# Agent: Lead Data Specialist
## Task: Collect and analyze the following information about the lead:
- Personal Information:
  - Name: Obtain the full name of the lead.
  - Job Title: Determine the lead's current job title.
  - Role Relevance: Assess how relevant the lead's role is to the decision-making process on a scale from 0 to 10.
  - Professional Background: Optionally, gather a brief description of the lead's professional background.

- Company Information:
  - Company Name: Identify the name of the company the lead works for.
  - Industry: Determine the industry in which the company operates.
  - Company Size: Estimate the size of the company in terms of employee count.
  - Revenue: If available, collect information on the annual revenue of the company.
  - Market Presence: Evaluate the company's market presence on a scale from 0 to 10.

- Our Company and Product:
  - Company Name: CrewAI
  - Product: Multi-Agent Orch



# Agent: Lead Scorer and Validator
## Final Answer: 
{
  "personal_info": {
    "name": "Olivia Robinson",
    "job_title": "Director of Engineering",
    "role_relevance": 9,
    "professional_background": "Olivia Robinson has experience in engineering leadership roles, contributing significantly to strategic planning and execution at her current and past positions."
  },
  "company_info": {
    "company_name": "Clearbit",
    "industry": "Software, Technology, Information and Internet, Custom Software & Technical Consulting, Business Intelligence, Data and Analytics",
    "company_size": 116,
    "revenue": 41.6,
    "market_presence": 7
  },
  "lead_score": {
    "score": 88,
    "scoring_criteria": [
      "Role Relevance: 9/10 - High relevance in decision-making",
      "Company Size: 8/10 - Mid-sized company with potential for growth",
      "Market Presence: 7/10 - Moderate presence with a specialized focus",
      "Cultural Fit: 9/10 - Values align well with strategic objecti

# Usage Metrics and Costs

In [18]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0073


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,48946,47015,29184,1931,14


In [19]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0002


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,1427,1102,0,325,2


# Inspecting Results

In [20]:
scores = flow.state["score_crews_results"]

In [22]:
import pandas as pd
from IPython.display import display, HTML

lead_scoring_result = scores[0].pydantic

# Create a dictionary with the nested structure flattened
data = {
    'Name': lead_scoring_result.personal_info.name,
    'Job Title': lead_scoring_result.personal_info.job_title,
    'Role Relevance': lead_scoring_result.personal_info.role_relevance,
    'Professional Background': lead_scoring_result.personal_info.professional_background,
    'Company Name': lead_scoring_result.company_info.company_name,
    'Industry': lead_scoring_result.company_info.industry,
    'Company Size': lead_scoring_result.company_info.company_size,
    'Revenue': lead_scoring_result.company_info.revenue,
    'Market Presence': lead_scoring_result.company_info.market_presence,
    'Lead Score': lead_scoring_result.lead_score.score,
    'Scoring Criteria': ', '.join(lead_scoring_result.lead_score.scoring_criteria),
    'Validation Notes': lead_scoring_result.lead_score.validation_notes
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Reset the index to turn the original column names into a regular column
df = df.reset_index()

# Rename the index column to 'Attribute'
df = df.rename(columns={'index': 'Attribute'})

# Create HTML table with bold attributes and left-aligned values
html_table = df.style.set_properties(**{'text-align': 'left'}) \
                     .format({'Attribute': lambda x: f'<b>{x}</b>'}) \
                     .hide(axis='index') \
                     .to_html()

# Display the styled HTML table
display(HTML(html_table))

Attribute,Value
Name,Olivia Robinson
Job Title,Director of Engineering
Role Relevance,9
Professional Background,"Olivia Robinson has experience in engineering leadership roles, contributing significantly to strategic planning and execution at her current and past positions."
Company Name,Clearbit
Industry,"Software, Technology, Information and Internet, Custom Software & Technical Consulting, Business Intelligence, Data and Analytics"
Company Size,116
Revenue,41.600000
Market Presence,7
Lead Score,88


In [23]:
import textwrap

result_text = emails[0].raw
wrapped_text = textwrap.fill(result_text, width=80)
print(wrapped_text)

Olivia,  Unlock new levels of efficiency and innovation with CrewAI's Multi-
Agent Orchestration Platform. As the Director of Engineering at Clearbit, your
expertise is pivotal in harnessing the power of AI for strategic planning and
execution. Imagine orchestrating AI agents tailored specifically to your needs,
streamlining your processes, and driving forward Clearbit's market presence.
Don't miss the opportunity to elevate Clearbit's capabilities. Dive deeper into
how CrewAI can align with your strategic objectives and cultural values.
Schedule a meeting with us today to explore the transformative potential of
CrewAI for Clearbit. Click here to book your session now!   Let's take the next
step in Clearbit's growth journey together.


In [37]:
from crewai import Flow
from crewai.flow.flow import listen, start, and_, or_, router

class SalesPipeline(Flow):

  @start()
  def fetch_leads(self):
    # Pull our leads from the database
    # This is a mock, in a real-world scenario, this is where you would
    # fetch leads from a database
    leads = [
      {
        "lead_data": {
          "name": "Keith Grey",
          "job_title": "Director of Engineering",
          "company": "Clearbit",
          "email": "keith_grey@clearbit.com",
          "use_case": "Using AI Agent to do better data enrichment."
        },
      },
    ]
    return leads

  @listen(fetch_leads)
  def score_leads(self, leads):
    scores = lead_scoring_crew.kickoff_for_each(leads)
    self.state["score_crews_results"] = scores
    return scores

  @listen(score_leads)
  def store_leads_score(self, scores):
    # Here we would store the scores in the database
    return scores

  @listen(score_leads)
  def filter_leads(self, scores):
    return [score for score in scores if score['lead_score'].score > 70]

  @listen(and_(filter_leads, store_leads_score))
  def log_leads(self, leads):
    print(f"Leads: {leads}")

  @router(filter_leads,)   # ["high", "medium", "low"]
  def count_leads(self, scores):
    if len(scores) > 10:
      return 'high'
    elif len(scores) > 5:
      return 'medium'
    else:
      return 'low'

  @listen('high')
  def store_in_salesforce(self, leads):
    return leads

  @listen('medium')
  def send_to_sales_team(self, leads):
    return leads

  @listen('low')
  def write_email(self, leads):
    scored_leads = [lead.to_dict() for lead in leads]
    emails = email_writing_crew.kickoff_for_each(scored_leads)
    return emails

  @listen(write_email)
  def send_email(self, emails):
    # Here we would send the emails to the leads
    return emails

flow = SalesPipeline()

# Plotting the flow

In [38]:
flow = SalesPipeline()
flow.plot()

Plot saved as crewai_flow.html


In [39]:
from IPython.display import HTML, display

with open('/content/crewai_flow.html', 'r') as f:
    html_content = f.read()

display(HTML(html_content))